In [1]:
import pandas as pd
import numpy as np
import ast
from pandas.io.json import json_normalize

In [2]:
df = pd.read_csv('..\Data\home-listing-details.csv')

In [3]:
df.head()

,listingid,fulladdress,cdp,primarydetails,secondarydetails,features,description,longitude,latitude
0,301640001252,"30937 Oakwood Road, Ocean View, DE 19970-3811",Denton Woods,"{""MLS#"": ""DESU138282"", ""Interior Sq. Ft"": ""1,9...","{""Garage"": ""N"", ""Condo Assoc"": ""No"", ""Basement...","{""Utilities"": ""Heating - None, Propane - Lease...",Check out this large home in the quiet communi...,-75.105615,38.562339
1,301201455280,"1076 Merchant Street, Coatesville, PA 19320-3344",Penn Crossing,"{""Acreage"": ""0.06"", ""Interior Sq. Ft"": ""1,244""...","{""Dining Room"": ""9 X 9 - Main"", ""Home Assoc"": ...","{""Community"": ""HOA: Yes; Fee Includes: Common ...",Beautiful Penn Crossing Townhome is now availa...,-75.805810,39.988335
2,301640104137,"10 Holywell Drive, Malvern, PA 19355",Atwater,"{""MLS#"": ""PACT475594"", ""Interior Sq. Ft"": ""1,7...","{""Bedroom 1"": ""Upper 1"", ""Condo Assoc"": ""No"", ...","{""Utilities"": ""Energy Star Heating System, Fo...",We've opened a new phase in the community - we...,-75.520000,40.050000
3,301070195213,"12 Cedarwood Drive, Lewes, DE 19958",Lochwood,"{""Age"": ""0"", ""Interior Sq. Ft"": ""1,200"", ""Acre...","{""Taxes"": ""$1,200"", ""Condo Assoc"": ""No"", ""Pool...","{""Garage/Parking"": ""Attached Garage, Driveway...",TO BE BUILT HOME IN THE CAPE HENLOPEN SCHOOL D...,-75.197289,38.676398
4,301507361366,"10113 Fast Colors Lane, Berlin, MD 21811",Glen Riddle,"{""Acreage"": ""0.23"", ""Design/Type"": ""Detached"",...","{""Family Room"": ""19 X 16 - Main"", ""High School...","{""Interior"": ""Carpet, Dining Area, Entry Lev...",GlenRiddle is a gated resort-style community w...,28.516670,47.250000


In [4]:
# Both from https://stackoverflow.com/a/39906235
def only_dict(d):
    return ast.literal_eval(d)

def list_of_dicts(ld):
    return dict([(list(d.values())[1], list(d.values())[0]) for d in ast.literal_eval(ld)])

In [5]:
pdeets = json_normalize(df['primarydetails'].apply(only_dict).tolist())

In [6]:
pdeets = pdeets.rename(columns={'# of Units':'UnitNum',
                                'Acreage':'Acres',
                                'Design/Type':'Design',
                                'Interior Sq. Ft':'intsqft',
                                'MLS#':'MLSNum',
                                'Mobile#':'MobileNum'})

In [7]:
pdeets.head()

,UnitNum,Acres,Age,Beds/Baths,Design,intsqft,MLSNum,MobileNum,Style,Subdivision,Taxes,Zoning
0,NaN,0.17,19,3/2,Detached,"1,960",DESU138282,NaN,Modular/Pre-Fabricated,Denton Woods,NaN,NaN
1,NaN,0.06,16,3/1.1,Row/Townhouse,"1,244",PACT212570,NaN,Traditional,Penn Crossing,NaN,NaN
2,NaN,0.02,1,3/2.1,Row/Townhouse,"1,765",PACT475594,NaN,"Contemporary, Traditional",Atwater,NaN,NaN
3,NaN,0.23,0,3/2,Detached,"1,200",DESU118054,NaN,Ranch/Rambler,Lochwood,NaN,NaN
4,NaN,0.23,0,4/3.1,Detached,"3,256",MDWO104018,NaN,Traditional,Glen Riddle,NaN,NaN


In [8]:
pdeets = (pdeets.join(pdeets['Beds/Baths']
                .str.split('/',expand=True)
                .rename(columns={0:'Beds', 1:'Baths'})))



In [9]:
pdeets = (pdeets.join(pdeets['Baths']
                .str.split('.',expand=True)
                .rename(columns={0:'FullBaths',1:'HalfBaths'})))

del pdeets['Beds/Baths']
del pdeets['Baths']

In [10]:
pdeets.head()

,UnitNum,Acres,Age,Design,intsqft,MLSNum,MobileNum,Style,Subdivision,Taxes,Zoning,Beds,FullBaths,HalfBaths
0,NaN,0.17,19,Detached,"1,960",DESU138282,NaN,Modular/Pre-Fabricated,Denton Woods,NaN,NaN,3,2,None
1,NaN,0.06,16,Row/Townhouse,"1,244",PACT212570,NaN,Traditional,Penn Crossing,NaN,NaN,3,1,1
2,NaN,0.02,1,Row/Townhouse,"1,765",PACT475594,NaN,"Contemporary, Traditional",Atwater,NaN,NaN,3,2,1
3,NaN,0.23,0,Detached,"1,200",DESU118054,NaN,Ranch/Rambler,Lochwood,NaN,NaN,3,2,None
4,NaN,0.23,0,Detached,"3,256",MDWO104018,NaN,Traditional,Glen Riddle,NaN,NaN,4,3,1


In [11]:
pdeets.loc[pdeets['MobileNum'].notnull(),'MobileNum'].head()

21     7598
57     7788
62     6948
68     7197
116    7386
Name: MobileNum, dtype: object

In [12]:
pdeets = pdeets.replace(r'^\s*$', np.nan, regex=True)

In [13]:
type_change_dict = {'Acres':float,
                    'Age':int,
                    'UnitNum':float,
                    'Design':'category',
                    'Style':'category',
                    'Beds':float,
                    'FullBaths':float,
                    'HalfBaths':float}
pdeets = pdeets.astype(type_change_dict)

In [14]:
pdeets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
UnitNum        14 non-null float64
Acres          1977 non-null float64
Age            2000 non-null int32
Design         1990 non-null category
intsqft        1842 non-null object
MLSNum         2000 non-null object
MobileNum      69 non-null object
Style          1967 non-null category
Subdivision    1983 non-null object
Taxes          13 non-null object
Zoning         19 non-null object
Beds           1986 non-null float64
FullBaths      1986 non-null float64
HalfBaths      1317 non-null float64
dtypes: category(2), float64(5), int32(1), object(6)
memory usage: 192.2+ KB


In [15]:
pdeets.loc[pd.to_numeric(pdeets['Acres']).idxmax()]

UnitNum                 NaN
Acres                  9500
Age                      30
Design             Detached
intsqft                 NaN
MLSNum           DESU138224
MobileNum               NaN
Style          Contemporary
Subdivision     Ocean Ridge
Taxes                   NaN
Zoning                  NaN
Beds                      6
FullBaths                 3
HalfBaths               NaN
Name: 12, dtype: object